In [10]:
# download data from https://www.kaggle.com/datasets/sachinpatel21/az-handwritten-alphabets-in-csv-format

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split as tts
import numpy as np 
import cv2 
from collections import deque 
import time
import os
from keras.models import load_model
import pandas as pd
import matplotlib.pyplot as plt
from keras.optimizers import Adadelta, SGD

training = input("Do you want to train a new model? Press (Y) for Yes else (N) for No")
alpha = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

def generate_model():
    model = Sequential([Conv2D(128,(3,3),activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1),padding='same'),
                    Conv2D(64,(3,3),activation='relu', kernel_initializer='he_uniform',padding='same'),
                    MaxPooling2D(2,2),
                    Conv2D(64,(3,3),activation='relu', kernel_initializer='he_uniform',padding='same'),
                    Conv2D(64,(3,3),activation='relu', kernel_initializer='he_uniform',padding='same'),
                    BatchNormalization(),
                    MaxPooling2D(2,2),
                    Flatten(),
                    Dense(100,activation='relu',kernel_initializer='he_uniform'),
                    Dropout(0.1),
                    Dense(64,activation='relu',kernel_initializer='he_uniform'),
                    Dropout(0.125),
                    BatchNormalization(),
                    Dense(26,activation='softmax')])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=SGD(learning_rate=0.01, momentum=0.9),
                  metrics=['accuracy'])
    model.summary() 
    return model

def train():
    
    df=pd.read_csv('a_z_handwritten_data.csv')
    data_array = np.array(df,dtype=np.uint8)
    del df  #memory issues
    
    labels = data_array[:,0]
    x = data_array[:,1:].reshape(372450,28,28)/255.
    del data_array
    unique, counts = np.unique(labels, return_counts=True)
    list_alpha = list(zip(alpha, counts))

    fig=plt.figure(figsize=(15,6))
    plt.xlabel('ALPHABETS',fontsize=14)
    plt.ylabel('COUNT',fontsize=14)
    plt.bar(alpha,counts)

    a = np.random.randint(low=0,high=372449,size=10)
    fig=plt.figure(figsize=(30,30))
    c=1
    for i in a:
        fig.add_subplot(20,20,c)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(x[i],cmap='gray')
        c+=1
        
    del a   # cleaning
    del c, list_alpha, counts, unique # cleaning

    x=x.reshape(372450,28,28,1)
    x_train,x_test,y_train,y_test = tts(x,labels,test_size=0.01)
    del x  # memory issues
    del labels

    model = generate_model()
    history = model.fit(x=x_train,y=y_train,validation_split=0.1,epochs=5)
    
    val_loss=history.history['val_loss']
    val_accuracy=history.history['val_accuracy']
    loss=history.history['loss']
    accuracy=history.history['accuracy']

    fig=plt.figure(figsize=(10,15))
    fig.add_subplot(2, 1, 1)
    plt.title('Cross Entropy Loss')
    plt.plot(loss, color='blue', label='train')
    plt.plot(val_loss, color='red', label='test')
    plt.legend()
    # plot accuracy
    fig.add_subplot(2, 1, 2)
    plt.title('Classification Accuracy')
    plt.plot(accuracy, color='blue', label='train')
    plt.plot(val_accuracy, color='red', label='test')
    plt.legend()

    metrics=model.evaluate(x_test,y_test)
    metrics

    model.save('Alphabet_Recognition.keras')

if training == 'Y':
    train()

model = load_model('Alphabet_Recognition.keras')

def color_detector():
    def setValues(x):
        print("")
    # creating a window for HUE selector
    cv2.namedWindow("Color detectors") 
    cv2.createTrackbar("Upper Hue", "Color detectors", 153, 180, setValues) 
    cv2.createTrackbar("Upper Saturation", "Color detectors", 255, 255, setValues) 
    cv2.createTrackbar("Upper Value", "Color detectors", 255, 255, setValues) 
    cv2.createTrackbar("Lower Hue", "Color detectors", 64, 180, setValues) 
    cv2.createTrackbar("Lower Saturation", "Color detectors", 72, 255, setValues) 
    cv2.createTrackbar("Lower Value", "Color detectors", 49, 255, setValues)

def alphabet_recognize(filepath):
    image = cv2.imread(filepath)
    blur_image=cv2.medianBlur(image,7)

    grey = cv2.cvtColor(blur_image, cv2.COLOR_BGR2GRAY)

    thresh = cv2.adaptiveThreshold(grey,200,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,41,25)

    contours,hierarchy= cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    preprocessed_digits = []

    # initialize the reverse flag and sort index
    # handle if we need to sort in reverse
    boundingBoxes = [cv2.boundingRect(c) for c in contours]
    (contours, boundingBoxes) = zip(*sorted(zip(contours, boundingBoxes),
                                    key=lambda b:b[1][0], reverse=False))


    for c in contours:
        x,y,w,h = cv2.boundingRect(c)

        # Creating a rectangle around the digit in the original image (for displaying the digits fetched via contours)
        cv2.rectangle(blur_image, (x,y), (x+w, y+h), color=(255, 0, 0), thickness=2)

        # Cropping out the digit from the image corresponding to the current contours in the for loop
        digit = thresh[y:y+h, x:x+w]

        # Resizing that digit to (18, 18)
        resized_digit = cv2.resize(digit, (18,18))

        # Padding the digit with 5 pixels of black color (zeros) in each side to finally produce the image of (28, 28)
        padded_digit = np.pad(resized_digit, ((5,5),(5,5)), "constant", constant_values=0)

        # Adding the preprocessed digit to the list of preprocessed digits
        preprocessed_digits.append(padded_digit)

    inp = np.array(preprocessed_digits)
    i=1
    alphabets=[]
    for digit in preprocessed_digits:
        [prediction] = model.predict(digit.reshape(1, 28, 28, 1)/255.)
        pred=alpha[np.argmax(prediction)]
        alphabets.append(pred)
        i+=1
        
    print("The Recognized Alphabets are : " ,*alphabets)
    return "".join(alphabets)


image_save_path = os.path.join(os.getcwd(), "last_frame.jpg")
color_detector()

bpoints = [deque(maxlen = 512)] 
gpoints = [deque(maxlen = 512)] 
ypoints = [deque(maxlen = 512)] 
rpoints = [deque(maxlen = 512)] 

# Now to mark the pointers in the above colour array we introduce some index values Which would mark their positions  

blue_index = 0
green_index = 0
yellow_index = 0
red_index = 0

# The kernel is used for dilation of contour

kernel = np.ones((5, 5)) 

# The ink colours for the drawing purpose 
 
colors = [(255, 0, 0), (0, 255, 0), (0, 225, 255), (0, 0, 255)] 
colorIndex = 0

# Setting up the drawing board AKA The canvas 

paintWindow = np.zeros((1500, 1500, 3)) + 255

cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE) 
 
cap = cv2.VideoCapture(0) 
prediction = 'NA'
pause = False
while True: 

    # Reading the camera frame 
    ret, frame = cap.read() 
    # For saving
    # out = cv2.VideoWriter("Paint-Window.mp4", cv2.VideoWriter_fourcc(*'XVID'), 1, (frame.shape[1], frame.shape[0]))
    
    # Flipping the frame to see same side of the user  
    frame = cv2.flip(frame, 1) 
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) 

    # Getting the new positions of the trackbar and setting the new HSV values 

    u_hue = cv2.getTrackbarPos("Upper Hue", "Color detectors") 
    u_saturation = cv2.getTrackbarPos("Upper Saturation", "Color detectors") 
    u_value = cv2.getTrackbarPos("Upper Value","Color detectors") 
    l_hue = cv2.getTrackbarPos("Lower Hue", "Color detectors") 
    l_saturation = cv2.getTrackbarPos("Lower Saturation", "Color detectors") 
    l_value = cv2.getTrackbarPos("Lower Value", "Color detectors") 
    Upper_hsv = np.array([u_hue, u_saturation, u_value]) 
    Lower_hsv = np.array([l_hue, l_saturation, l_value]) 

    # Adding the colour buttons to the live frame to choose color
    frame = cv2.rectangle(frame, (35, 1), (135, 65), (122, 122, 122), -1) 
    frame = cv2.rectangle(frame, (160, 1), (255, 65), (255, 0, 0), -1) 
    frame = cv2.rectangle(frame, (275, 1), (370, 65), (0, 255, 0), -1) 
    frame = cv2.rectangle(frame, (390, 1), (485, 65), (0, 255, 255), -1) 
    frame = cv2.rectangle(frame, (505, 1), (600, 65), (0, 0, 255), -1) 

    cv2.putText(frame, "Clear All", (55, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 2, cv2.LINE_AA) 

    cv2.putText(frame, "Blue Color", (175, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 2, cv2.LINE_AA) 
    
    cv2.putText(frame, "Green Color", (285, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 2, cv2.LINE_AA) 

    cv2.putText(frame, "Yellow Color", (400, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (150, 150, 150), 2, cv2.LINE_AA) 

    cv2.putText(frame, "Red Color", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 2, cv2.LINE_AA) 


    # masking out the pointer for it's identification in the frame 

    Mask = cv2.inRange(hsv, Lower_hsv, Upper_hsv) 
    Mask = cv2.erode(Mask, kernel, iterations = 1) 
    Mask = cv2.morphologyEx(Mask, cv2.MORPH_OPEN, kernel) 
    Mask = cv2.dilate(Mask, kernel, iterations = 1) 

    # Now contouring the pointers post identification 
    lower_red = np.array([100,60,60])
    upper_red = np.array([140,255,255])
    red_mask = cv2.inRange(hsv, lower_red, upper_red)
    red_mask = cv2.erode(red_mask, kernel, iterations=1)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)
    red_mask = cv2.dilate(red_mask, kernel, iterations=1)

    countours = []
    if pause == False:
        countours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        centre = None

    # If there are any contours formed 
    if len(countours) > 0 and pause == False: 
        
        # sorting the contours for the biggest 
        countour = sorted(countours, key = cv2.contourArea, reverse = True)[0] 
        # Get the radius of the cirlce formed around the found contour   
        ((x, y), radius) = cv2.minEnclosingCircle(countour) 
        
        # Drawing the circle boundary around the contour 
        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2) 
        
        # Calculating the centre of the detected contour 
        M = cv2.moments(countour) 
        centre = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00'])) 
        
        # Now checking if the user clicked on another button on the screen (the 4 buttons that were mentioned Y,G,B,R and clear all)
        if centre[1] <= 65: 
            
            # Clear Button 
            if 35 <= centre[0] <= 135: 
                bpoints = [deque(maxlen = 512)] 
                gpoints = [deque(maxlen = 512)] 
                ypoints = [deque(maxlen = 512)] 
                rpoints = [deque(maxlen = 512)] 

                blue_index = 0
                green_index = 0
                yellow_index = 0
                red_index = 0

                paintWindow[67:, :, :] = 255
            elif 160 <= centre[0] and centre[0] <= 255: 
                colorIndex = 0 # Blue 
                    
            elif 275 <= centre[0] and centre[0] <= 370: 
                colorIndex = 1 # Green 
            elif 390 <= centre[0] and centre[0] <= 485: 
                colorIndex = 2 # Yellow
            elif 505 <= centre[0] and centre[0] <= 600: 
                colorIndex = 3 # Red 
        else : 
            if colorIndex == 0: 
                bpoints[blue_index].appendleft(centre) 
            elif colorIndex == 1: 
                gpoints[green_index].appendleft(centre) 
            elif colorIndex == 2: 
                ypoints[yellow_index].appendleft(centre) 
            elif colorIndex == 3: 
                rpoints[red_index].appendleft(centre) 
                
    # Appending the next deques if nothing is detected

    else: 
        bpoints.append(deque(maxlen = 512)) 
        blue_index += 1
        gpoints.append(deque(maxlen = 512)) 
        green_index += 1
        ypoints.append(deque(maxlen = 512)) 
        yellow_index += 1
        rpoints.append(deque(maxlen = 512)) 
        red_index += 1

    # Drawing the lines of every colour on the canvas and the track frame window
    
    points = [bpoints, gpoints, ypoints, rpoints] 
    for i in range(len(points)): 
        for j in range(len(points[i])): 
            for k in range(1, len(points[i][j])): 
                if points[i][j][k - 1] is None or points[i][j][k] is None: 
                    continue
                    
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 30) 
                # cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 30)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], (0, 0, 0), 30)

    key = cv2.waitKey(1)    

    # letter_predictor = LetterPredictor()        
    if key & 0xFF == ord('v'):          
        cv2.imwrite(image_save_path, paintWindow)
        prediction = alphabet_recognize(image_save_path)

    if key & 0xFF == ord('x'):          
        pause = not pause

    # letter_predictor = LetterPredictor()        
    if key & 0xFF == ord('c'):           
        bpoints = [deque(maxlen = 512)] 
        gpoints = [deque(maxlen = 512)] 
        ypoints = [deque(maxlen = 512)] 
        rpoints = [deque(maxlen = 512)] 
        blue_index = 0
        green_index = 0
        yellow_index = 0
        red_index = 0
        paintWindow[:, :, :] = 255
        
    cv2.putText(frame, "Prediction: " + str(prediction), (20,800), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,102,0), 8)
    cv2.putText(frame, "V to predict", (1500,400), cv2.FONT_HERSHEY_SIMPLEX, 2, (102,51,0), 8)
    cv2.putText(frame, "C to Clear", (1500,500), cv2.FONT_HERSHEY_SIMPLEX, 2, (102,51,0), 8)
    cv2.putText(frame, "Q to Quit", (1500,600), cv2.FONT_HERSHEY_SIMPLEX, 2, (102,51,0), 8)
    
    # Displaying/running all the 3 windows 
    cv2.imshow("Live Tracking", frame) 
    cv2.imshow("Paint", paintWindow) 
    
    # For quitting/breaking the loop - press and hold ctrl+q twice 
    if key & 0xFF == ord("q"): 
        break

# Releasing the camera and all the other resources of the device  
cap.release() 
cv2.destroyAllWindows() 
key = cv2.waitKey(1) 

Do you want to train a new model? Press (Y) for Yes else (N) for No N
